In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models,layers,Sequential
import matplotlib.pyplot as plt

In [3]:
isi = 256
bs = 32
ch = 3
epochs  = 50

In [4]:
data = tf.keras.preprocessing.image_dataset_from_directory(
    'potato',
shuffle = True,
image_size = (isi,isi),
batch_size = bs)

Found 2152 files belonging to 3 classes.


In [5]:
class_name = data.class_names
class_name

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [10]:
len(data)

68

In [6]:
train_ds = data.take(54)

In [7]:
test_ds = data.skip(54)

In [8]:
val_ds = test_ds.take(6)

In [9]:
test_ds = test_ds.skip(6)

In [10]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [11]:
resize = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(isi,isi),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [12]:
argumentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [13]:
inpu = (bs,isi,isi,ch)
n_class = 3
model =  models.Sequential([
    resize,
    argumentation,
    layers.Conv2D(32,(3,3), activation = 'relu' , input_shape  = inpu),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size = (3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)) ,
    layers.Conv2D(64,kernel_size = (3,3), activation = 'relu'),
    layers.MaxPooling2D((3,3)),
    layers.Conv2D(64,(3,3), activation = 'relu' ),
    layers.MaxPooling2D((3,3)) ,
    layers.Conv2D(64,(3,3), activation = 'relu'),
    layers.MaxPooling2D((3,3)) ,
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_class,activation='softmax')
])

model.build(input_shape = inpu)

In [14]:
model.compile(optimizer = 'adam',
             loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
             metrics = ['accuracy'])

In [ ]:
model.fit(
train_ds,
epochs = 3,
batch_size=bs,
validation_data = val_ds)

Epoch 1/3
